Step by step visualisation of everything

In [3]:
import numpy as np
import pandas as pd
import datetime as dt

In [1]:
from helpers.util import connect
con, cur = connect('vertica')

from helpers.vertica import create_cases, add_aggr_nvl, check_tree_for_instance, check_tree_for_replace_dividers

In [4]:
formulas = pd.read_sql('''select KPI_SHORT, VENDOR, FORMULA_NUM_DENOM, DATE_START from VERTICA_STAT.FORMULA where KPI_SHORT in ('UE_THROUGHPUT_DL', 'UE_THROUGHPUT_UL') ''', con, parse_dates=['DATE_START'])
num_den = pd.read_sql('''select upper(NUM_DEN) NUM_DEN, VENDOR, FORMULA, DATE_START, AG, STANDART from VERTICA_STAT.NUM_DENOM where NUM_DEN in ('NUE_THROUGHPUT_DL_KBIT', 'NUE_THROUGHPUT_UL_KBIT', 'dUE_Throughput_DL', 'dUE_Throughput_UL') and vendor = 'N' ''', con, parse_dates=['DATE_START']) 

In [5]:
formulas

,KPI_SHORT,VENDOR,FORMULA_NUM_DENOM,DATE_START
0,UE_THROUGHPUT_DL,N,[NUE_THROUGHPUT_DL_KBIT / DUE_THROUGHPUT_DL],2018-01-01
1,UE_THROUGHPUT_UL,N,[NUE_THROUGHPUT_UL_KBIT / DUE_THROUGHPUT_UL],2018-01-01


In [6]:
num_den

,NUM_DEN,VENDOR,FORMULA,DATE_START,AG,STANDART
0,NUE_THROUGHPUT_UL_KBIT,N,(8*((M8012C91+M8012C93+M8012C95+M8012C97+M8012...,2018-01-01,SUM,4G
1,DUE_THROUGHPUT_UL,N,((M8012C156+M8012C157+M8012C158+M8012C159+M801...,2018-01-01,SUM,4G
2,NUE_THROUGHPUT_DL_KBIT,N,(8*((M8012C117+M8012C119+M8012C121+M8012C123+M...,2018-01-01,SUM,4G
3,DUE_THROUGHPUT_DL,N,((M8012C118+M8012C120+M8012C122+M8012C124+M801...,2018-01-01,SUM,4G
4,DUE_THROUGHPUT_DL,N,(M8012C165+M8012C166+M8012C167+M8012C168+M8012...,2021-03-10,SUM,4G


In [7]:
num_den['FIXED_FORMULAS'] = [add_aggr_nvl(i[1]['FORMULA'], i[1]['AG']) for i in num_den.iterrows()]

In [10]:
num_den.loc[0, 'NUM_DEN'], num_den.loc[0, 'FORMULA'], add_aggr_nvl(num_den.loc[0, 'FORMULA'], num_den.loc[0, 'AG'])

('NUE_THROUGHPUT_UL_KBIT',
 '(8*((M8012C91+M8012C93+M8012C95+M8012C97+M8012C99+M8012C101+M8012C103+M8012C105+M8012C107)/(8*1000)))',
 '(8*(((((((((SUM(NVL(M8012C91, 0))+SUM(NVL(M8012C93, 0)))+SUM(NVL(M8012C95, 0)))+SUM(NVL(M8012C97, 0)))+SUM(NVL(M8012C99, 0)))+SUM(NVL(M8012C101, 0)))+SUM(NVL(M8012C103, 0)))+SUM(NVL(M8012C105, 0)))+SUM(NVL(M8012C107, 0)))/(8*1000)))')

In [11]:
num_den

,NUM_DEN,VENDOR,FORMULA,DATE_START,AG,STANDART,FIXED_FORMULAS
0,NUE_THROUGHPUT_UL_KBIT,N,(8*((M8012C91+M8012C93+M8012C95+M8012C97+M8012...,2018-01-01,SUM,4G,"(8*(((((((((SUM(NVL(M8012C91, 0))+SUM(NVL(M801..."
1,DUE_THROUGHPUT_UL,N,((M8012C156+M8012C157+M8012C158+M8012C159+M801...,2018-01-01,SUM,4G,"(((((((((SUM(NVL(M8012C156, 0))+SUM(NVL(M8012C..."
2,NUE_THROUGHPUT_DL_KBIT,N,(8*((M8012C117+M8012C119+M8012C121+M8012C123+M...,2018-01-01,SUM,4G,"(8*(((((((((SUM(NVL(M8012C117, 0))+SUM(NVL(M80..."
3,DUE_THROUGHPUT_DL,N,((M8012C118+M8012C120+M8012C122+M8012C124+M801...,2018-01-01,SUM,4G,"(((((((((SUM(NVL(M8012C118, 0))+SUM(NVL(M8012C..."
4,DUE_THROUGHPUT_DL,N,(M8012C165+M8012C166+M8012C167+M8012C168+M8012...,2021-03-10,SUM,4G,"(((((((((SUM(NVL(M8012C165, 0))+SUM(NVL(M8012C..."


In [12]:
col1 = 'NUM_DEN' # source of counters in DATA
col2 = 'FIXED_FORMULAS' # source of formulas in DATA
counters = create_cases(col1, col2, num_den)

In [13]:
counters

{'NUE_THROUGHPUT_DL_KBIT': '(8*(((((((((SUM(NVL(M8012C117, 0))+SUM(NVL(M8012C119, 0)))+SUM(NVL(M8012C121, 0)))+SUM(NVL(M8012C123, 0)))+SUM(NVL(M8012C125, 0)))+SUM(NVL(M8012C127, 0)))+SUM(NVL(M8012C129, 0)))+SUM(NVL(M8012C131, 0)))+SUM(NVL(M8012C133, 0)))/(8*1000)))',
 'DUE_THROUGHPUT_UL': '(((((((((SUM(NVL(M8012C156, 0))+SUM(NVL(M8012C157, 0)))+SUM(NVL(M8012C158, 0)))+SUM(NVL(M8012C159, 0)))+SUM(NVL(M8012C160, 0)))+SUM(NVL(M8012C161, 0)))+SUM(NVL(M8012C162, 0)))+SUM(NVL(M8012C163, 0)))+SUM(NVL(M8012C164, 0)))/1000)',
 'NUE_THROUGHPUT_UL_KBIT': '(8*(((((((((SUM(NVL(M8012C91, 0))+SUM(NVL(M8012C93, 0)))+SUM(NVL(M8012C95, 0)))+SUM(NVL(M8012C97, 0)))+SUM(NVL(M8012C99, 0)))+SUM(NVL(M8012C101, 0)))+SUM(NVL(M8012C103, 0)))+SUM(NVL(M8012C105, 0)))+SUM(NVL(M8012C107, 0)))/(8*1000)))',
 'DUE_THROUGHPUT_DL': {'case': {'when': [[{'between': {'counter': 'datetime',
       'date_start': datetime.date(2021, 3, 10),
       'date_end': datetime.date(2021, 3, 12),
       'by_date': True}},
     '((((((((

In [14]:
num_den['CASE'] = [counters[i] for i in num_den['NUM_DEN']]

In [15]:
num_den

,NUM_DEN,VENDOR,FORMULA,DATE_START,AG,STANDART,FIXED_FORMULAS,CASE
0,NUE_THROUGHPUT_UL_KBIT,N,(8*((M8012C91+M8012C93+M8012C95+M8012C97+M8012...,2018-01-01,SUM,4G,"(8*(((((((((SUM(NVL(M8012C91, 0))+SUM(NVL(M801...","(8*(((((((((SUM(NVL(M8012C91, 0))+SUM(NVL(M801..."
1,DUE_THROUGHPUT_UL,N,((M8012C156+M8012C157+M8012C158+M8012C159+M801...,2018-01-01,SUM,4G,"(((((((((SUM(NVL(M8012C156, 0))+SUM(NVL(M8012C...","(((((((((SUM(NVL(M8012C156, 0))+SUM(NVL(M8012C..."
2,NUE_THROUGHPUT_DL_KBIT,N,(8*((M8012C117+M8012C119+M8012C121+M8012C123+M...,2018-01-01,SUM,4G,"(8*(((((((((SUM(NVL(M8012C117, 0))+SUM(NVL(M80...","(8*(((((((((SUM(NVL(M8012C117, 0))+SUM(NVL(M80..."
3,DUE_THROUGHPUT_DL,N,((M8012C118+M8012C120+M8012C122+M8012C124+M801...,2018-01-01,SUM,4G,"(((((((((SUM(NVL(M8012C118, 0))+SUM(NVL(M8012C...",{'case': {'when': [[{'between': {'counter': 'd...
4,DUE_THROUGHPUT_DL,N,(M8012C165+M8012C166+M8012C167+M8012C168+M8012...,2021-03-10,SUM,4G,"(((((((((SUM(NVL(M8012C165, 0))+SUM(NVL(M8012C...",{'case': {'when': [[{'between': {'counter': 'd...


In [16]:
def find_end_date(data, dates, kpi):
    temp = formulas.loc[data['KPI_SHORT']==kpi]
    temp = temp['DATE_START']
    temp = sorted(temp)
    ind = temp.index(dates)
    if ind < len(temp)-1:
        return(temp[ind+1].date()-dt.timedelta(days=1) )
    else:
        return(dt.datetime.today().date()) 

In [17]:
formulas.loc[0, 'KPI_SHORT'], formulas.loc[0,'DATE_START'], find_end_date(formulas, formulas.loc[0,'DATE_START'], formulas.loc[0, 'KPI_SHORT'])

('UE_THROUGHPUT_DL',
 Timestamp('2018-01-01 00:00:00'),
 datetime.date(2021, 3, 11))

In [18]:
formulas['END_DATE'] = [find_end_date(formulas, i[1]['DATE_START'], i[1]['KPI_SHORT']) for i in formulas.iterrows()]

In [19]:
formulas

,KPI_SHORT,VENDOR,FORMULA_NUM_DENOM,DATE_START,END_DATE
0,UE_THROUGHPUT_DL,N,[NUE_THROUGHPUT_DL_KBIT / DUE_THROUGHPUT_DL],2018-01-01,2021-03-11
1,UE_THROUGHPUT_UL,N,[NUE_THROUGHPUT_UL_KBIT / DUE_THROUGHPUT_UL],2018-01-01,2021-03-11


In [21]:
formulas.loc[0,'FORMULA_NUM_DENOM'], formulas.loc[0,'DATE_START'].date(), formulas.loc[0,'END_DATE'], check_tree_for_replace_dividers(formulas.loc[0,'FORMULA_NUM_DENOM'], formulas.loc[0,'DATE_START'], formulas.loc[0,'END_DATE'], counters)

('[NUE_THROUGHPUT_DL_KBIT / DUE_THROUGHPUT_DL]',
 datetime.date(2018, 1, 1),
 datetime.date(2021, 3, 11),
 "ONLINESTAT.F_DEV((8*(((((((((SUM(NVL(M8012C117, 0))+SUM(NVL(M8012C119, 0)))+SUM(NVL(M8012C121, 0)))+SUM(NVL(M8012C123, 0)))+SUM(NVL(M8012C125, 0)))+SUM(NVL(M8012C127, 0)))+SUM(NVL(M8012C129, 0)))+SUM(NVL(M8012C131, 0)))+SUM(NVL(M8012C133, 0)))/(8*1000))), case when datetime between to_date('2021-03-10', 'YYYY-MM-DD') and to_date('2021-03-11', 'YYYY-MM-DD') then (((((((((SUM(NVL(M8012C165, 0))+SUM(NVL(M8012C166, 0)))+SUM(NVL(M8012C167, 0)))+SUM(NVL(M8012C168, 0)))+SUM(NVL(M8012C169, 0)))+SUM(NVL(M8012C170, 0)))+SUM(NVL(M8012C171, 0)))+SUM(NVL(M8012C172, 0)))+SUM(NVL(M8012C173, 0)))/1000) when datetime between to_date('2018-01-01', 'YYYY-MM-DD') and to_date('2021-03-09', 'YYYY-MM-DD') then (((((((((SUM(NVL(M8012C118, 0))+SUM(NVL(M8012C120, 0)))+SUM(NVL(M8012C122, 0)))+SUM(NVL(M8012C124, 0)))+SUM(NVL(M8012C126, 0)))+SUM(NVL(M8012C128, 0)))+SUM(NVL(M8012C130, 0)))+SUM(NVL(M8012C132, 

In [22]:
formulas['FIXED_FORMULAS'] = [check_tree_for_replace_dividers(i[1]['FORMULA_NUM_DENOM'],
                                                                i[1]['DATE_START'], i[1]['END_DATE'],
                                                                counters) for i in formulas.iterrows()]

In [23]:
formulas

,KPI_SHORT,VENDOR,FORMULA_NUM_DENOM,DATE_START,END_DATE,FIXED_FORMULAS
0,UE_THROUGHPUT_DL,N,[NUE_THROUGHPUT_DL_KBIT / DUE_THROUGHPUT_DL],2018-01-01,2021-03-11,ONLINESTAT.F_DEV((8*(((((((((SUM(NVL(M8012C117...
1,UE_THROUGHPUT_UL,N,[NUE_THROUGHPUT_UL_KBIT / DUE_THROUGHPUT_UL],2018-01-01,2021-03-11,"ONLINESTAT.F_DEV((8*(((((((((SUM(NVL(M8012C91,..."
